<a href="https://colab.research.google.com/github/tinawong15/portends-greatness/blob/master/elections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import pandas as pd

In [4]:
csv_path = os.path.join('countypres_2000-2016.csv') 
df = pd.read_csv(csv_path, sep="\t") 
df

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,1001.0,President,Al Gore,democrat,4942.0,17208.0,20181011
1,2000,Alabama,AL,Autauga,1001.0,President,George W. Bush,republican,11993.0,17208.0,20181011
2,2000,Alabama,AL,Autauga,1001.0,President,Ralph Nader,green,160.0,17208.0,20181011
3,2000,Alabama,AL,Autauga,1001.0,President,Other,NaN,113.0,17208.0,20181011
4,2000,Alabama,AL,Baldwin,1003.0,President,Al Gore,democrat,13997.0,56480.0,20181011
5,2000,Alabama,AL,Baldwin,1003.0,President,George W. Bush,republican,40872.0,56480.0,20181011
6,2000,Alabama,AL,Baldwin,1003.0,President,Ralph Nader,green,1033.0,56480.0,20181011
7,2000,Alabama,AL,Baldwin,1003.0,President,Other,NaN,578.0,56480.0,20181011
8,2000,Alabama,AL,Barbour,1005.0,President,Al Gore,democrat,5188.0,10395.0,20181011
9,2000,Alabama,AL,Barbour,1005.0,President,George W. Bush,republican,5096.0,10395.0,20181011
